#CREATE THE WEATHER DATA

##READ AND MERGE THE WEATHER DATA IN ONE DATASET

In [38]:
import os
import pandas as pd


folder_path = 'C:/Users/stili/OneDrive/Desktop/Data_sets/Data_sets/WeatherF'

# List all files in the directory
files = os.listdir(folder_path)
file_dict = {
    'cld.txt': 'Cloud cover',
    'dtr.txt': 'Diurnal Temperature Range',
    'frs.txt': 'Frost days',
    'pet.txt': 'potential evapo-transpiration',
    'pre.txt': 'Precipitation rate',
    'tmn.txt': 'Minimum 2m temperature',
    'tmp.txt': 'Mean 2m temperature',
    'tmx.txt': 'Maximum 2m temperature',
    'vap.txt': 'Vapour pressure',
    'wet.txt': 'Wet days',
}
data_frames = []
for file in files:

  df = pd.read_csv( folder_path+'/'+file, delim_whitespace=True)

  # Drop the last 5 columns
  df = df.iloc[:, :-5]

  # Melt the DataFrame to reshape
  melted_df = df.melt(id_vars=['YEAR'], var_name='Month', value_name= file_dict[file])

  # Create the 'Month-Year' column
  melted_df['Month-Year'] = melted_df['Month'] + '-' + melted_df['YEAR'].astype(str)

  # Convert Months to a categorical attribute to be used for sorting later. If i had theme like 01,02,03 etc this step is not needed'
  melted_df['Month'] = pd.Categorical(melted_df['Month'],
                                      categories=['JAN', 'FEB', 'MAR', 'APR', 'MAY',
                                                  'JUN', 'JUL', 'AUG', 'SEP', 'OCT',
                                                  'NOV', 'DEC'],
                                      ordered=True)
  # Sort the DataFrame
  final_df = melted_df.sort_values(by=['YEAR', 'Month'])

  # Select relevant columns
  final_df_clc = final_df[['Month-Year', file_dict[file]]]

  # Reset the index if needed
  final_df_clc.reset_index(drop=True, inplace=True)
  data_frames.append(final_df_clc)




C:\Users\stili\AppData\Local\Temp\ipykernel_5192\4221945506.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv( folder_path+'/'+file, delim_whitespace=True)
C:\Users\stili\AppData\Local\Temp\ipykernel_5192\4221945506.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv( folder_path+'/'+file, delim_whitespace=True)
C:\Users\stili\AppData\Local\Temp\ipykernel_5192\4221945506.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv( folder_path+'/'+file, delim_whitespace=True)
C:\Users\stili\AppData\Local\Temp\ipykernel_5192\4221945506.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future vers

In [39]:
merged_df = data_frames[0]
for df in data_frames[1:]:
    merged_df = pd.merge(merged_df, df, on='Month-Year', how='inner')  # 'outer' to keep all dates, adjust to 'inner' if you only want common dates

merged_df=merged_df.drop(columns="Diurnal Temperature Range")
merged_df.to_csv('C:/Users/stili/OneDrive/Desktop/Data_sets/Data_sets/merged_data.csv', index=False)

##TRANSFORM THE MONTHLY DATA OF THE WEATHER INTO YEARLY (AVERAGE FOR EACH YEAR)

In [40]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
file_path = 'C:/Users/stili/OneDrive/Desktop/Data_sets/Data_sets/merged_data.csv'
df = pd.read_csv(file_path)

# Preview the first few rows of the dataframe
df.head()


,Month-Year,Cloud cover,Frost days,potential evapo-transpiration,Precipitation rate,Minimum 2m temperature,Mean 2m temperature,Maximum 2m temperature,Vapour pressure,Wet days
0,JAN-1901,80.8,24.9,0.0,40.1,-3.7,-0.8,2.1,6.1,16.1
1,FEB-1901,77.3,23.1,0.1,29.3,-4.3,-1.2,1.9,5.7,11.0
2,MAR-1901,87.1,15.8,0.9,52.8,0.8,3.7,6.6,7.0,17.7
3,APR-1901,71.5,6.9,2.1,66.9,4.2,8.5,12.9,8.3,17.3
4,MAY-1901,53.5,1.9,3.1,29.3,7.1,12.5,18.0,10.9,11.0


###FILTER FROM 1961 AND LATES AND CALCULATE THE AVERAGE

In [44]:
# Extract the year from the 'Month-Year' column
df['Year'] = df['Month-Year'].str.extract(r'(\d{4})').astype(int)

# Filter the data to keep only records from 1961 and onwards
df_filtered = df[(df['Year'] >= 1961) & (df['Year'] <= 2022)]

# Drop the 'Month-Year' column as it's no longer needed
df_filtered = df_filtered.drop(columns=['Month-Year'])




# Define the aggregation functions for each column
agg_funcs = {col: 'mean' for col in df_filtered.columns if col not in [df_filtered.columns[2], df_filtered.columns[9]]}  # Mean for all except 2 and 9
agg_funcs[df_filtered.columns[2]] = 'sum'  # Sum for column 2
agg_funcs[df_filtered.columns[9]] = 'sum'  # Sum for column 9


# Group by 'Year' and apply the aggregation functions
df_yearly_avg = df_filtered.groupby('Year').agg(agg_funcs)
df_yearly_avg = df_yearly_avg[['Year'] + [col for col in df_yearly_avg.columns if col != 'Year']]
# Display the first few rows of the final DataFrame

df_yearly_avg.to_csv('C:/Users/stili/OneDrive/Desktop/Data_sets/Data_sets/yearly_average_merged_data.csv', index=False)


###ADD ALSO THE AVERAGE VALUES PES SEASON FOR EACH ATTRIBUTE PER EACH YEAR

In [48]:
import pandas as pd
import os
import glob

# Load the yearly_average_merged_data.csv
yearly_data = pd.read_csv('C:/Users/stili/OneDrive/Desktop/Data_sets/Data_Sets/yearly_average_merged_data.csv')

# Path to the folder containing the .txt files
folder_path = r'C:/Users/stili/OneDrive/Desktop/Data_sets/Data_Sets/WeatherF'

# Get all .txt files in the folder
txt_files = glob.glob(os.path.join(folder_path, '*.txt'))

# Iterate over each .txt file
for txt_file in txt_files:
    # Extract the filename without extension
    filename = os.path.splitext(os.path.basename(txt_file))[0]

    # Load the .txt file into a DataFrame
    wet_data = pd.read_csv(txt_file, delim_whitespace=True)

    # Select the columns 'YEAR', 'MAM', 'JJA', 'SON', and 'DJF'
    if {'YEAR', 'MAM', 'JJA', 'SON', 'DJF'}.issubset(wet_data.columns):
        wet_columns = wet_data[['YEAR', 'MAM', 'JJA', 'SON', 'DJF']]

        # Merge with the yearly_data on 'Year'
        yearly_data = pd.merge(yearly_data, wet_columns, how='left', left_on='Year', right_on='YEAR')

        # Rename the new columns to include the filename (e.g., 'wet_MAM')
        yearly_data = yearly_data.rename(columns={
            'MAM': f'{filename}_MAM',
            'JJA': f'{filename}_JJA',
            'SON': f'{filename}_SON',
            'DJF': f'{filename}_DJF'
        })

        # Drop the 'YEAR' column as it's now redundant
        yearly_data = yearly_data.drop(columns=['YEAR'])
    else:
        print(f"Skipping {txt_file}, required columns not found.")

# Save the updated yearly data to a new CSV file
output_file = folder_path+'/yearly_average_merged_data_with_all_txt.csv'
yearly_data.to_csv(output_file, index=False)

print(f"All .txt files processed and merged. Output saved to {output_file}")


All .txt files processed and merged. Output saved to C:/Users/stili/OneDrive/Desktop/Data_sets/Data_Sets/WeatherF/yearly_average_merged_data_with_all_txt.csv


C:\Users\stili\AppData\Local\Temp\ipykernel_5192\3662304556.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  wet_data = pd.read_csv(txt_file, delim_whitespace=True)
C:\Users\stili\AppData\Local\Temp\ipykernel_5192\3662304556.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  wet_data = pd.read_csv(txt_file, delim_whitespace=True)
C:\Users\stili\AppData\Local\Temp\ipykernel_5192\3662304556.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  wet_data = pd.read_csv(txt_file, delim_whitespace=True)
C:\Users\stili\AppData\Local\Temp\ipykernel_5192\3662304556.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`

##RETRIVE THE DATA FROM API FOR DAYLIGHT DURATION AND WIND SPEED

In [49]:
from geopy.geocoders import Nominatim
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import time


# Step 1: Create a geolocator object
geolocator = Nominatim(user_agent="my_geocoder_app", timeout=10)

# Step 2: Define the list of cities in the Netherlands
cities = [
    "Amsterdam", "Rotterdam", "The Hague", "Utrecht", "Eindhoven",
    "Groningen", "Tilburg", "Almere", "Breda", "Nijmegen",
    "Enschede", "Haarlem", "Haarlemmermeer", "Arnhem", "Zaanstad",
    "Amersfoort", "Apeldoorn", "Dordrecht", "Leiden", "Zoetermeer",
    "Emmen", "Venlo", "Leeuwarden", "Zwolle", "Helmond"
]

# Function to get weather data for a specific city
def get_weather_data(city_name, start_date="1961-01-01", end_date="2022-12-31"):
    # Get the location
    location = geolocator.geocode(f"{city_name}, Netherlands")

    if location:
        latitude = location.latitude
        longitude = location.longitude
        print(f"{city_name}: Latitude: {latitude}, Longitude: {longitude}")

        # Setup the Open-Meteo API client with cache and retry on error
        cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
        retry_session = retry(cache_session, retries=10, backoff_factor=0.2)
        openmeteo = openmeteo_requests.Client(session=retry_session)

        # Define API parameters
        url = "https://archive-api.open-meteo.com/v1/archive"
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": start_date,
            "end_date": end_date,
            "daily": ["daylight_duration", "wind_speed_10m_max"]
        }

        # Fetch weather data
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        # Process daily data
        daily = response.Daily()
        daily_daylight_duration = daily.Variables(0).ValuesAsNumpy()
        daily_wind_speed_10m_max = daily.Variables(1).ValuesAsNumpy()

        daily_data = {
            "date": pd.date_range(
                start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=daily.Interval()),
                inclusive="left"
            ),
            "daylight_duration": daily_daylight_duration,
            "wind_speed_10m_max": daily_wind_speed_10m_max
        }

        # Create DataFrame for the city
        daily_dataframe = pd.DataFrame(data=daily_data)
        daily_dataframe['city'] = city_name  # Add city name as a column
        return daily_dataframe
    else:
        print(f"Location for {city_name} not found.")
        return pd.DataFrame()  # Return an empty DataFrame

# Step 3: Collect data for each city and merge DataFrames
all_dataframes = []  # Initialize a list to hold DataFrames
for city in cities:
    time.sleep(65)  # Add a 1-second delay between requests
    city_data = get_weather_data(city)
    if not city_data.empty:  # Check if the DataFrame is not empty
        all_dataframes.append(city_data)  # Append the DataFrame to the list

# Merge all DataFrames by appending rows
merged_data = pd.concat(all_dataframes, ignore_index=True)
merged_data.to_csv('C:/Users/stili/OneDrive/Desktop/weather_data_with_Allcities.csv', index=False)
# Display the merged DataFrame
print(merged_data.head(5))


Amsterdam: Latitude: 52.3730796, Longitude: 4.8924534
Rotterdam: Latitude: 51.9244424, Longitude: 4.47775
The Hague: Latitude: 52.0799838, Longitude: 4.3113461
Utrecht: Latitude: 52.0907006, Longitude: 5.1215634
Eindhoven: Latitude: 51.4392648, Longitude: 5.478633
Groningen: Latitude: 53.2190652, Longitude: 6.5680077
Tilburg: Latitude: 51.58561845, Longitude: 5.0660616131367675
Almere: Latitude: 52.44175125, Longitude: 5.2092943104327585
Breda: Latitude: 51.5887845, Longitude: 4.7760237
Nijmegen: Latitude: 51.8474946, Longitude: 5.8637771
Enschede: Latitude: 52.2209855, Longitude: 6.8940537
Haarlem: Latitude: 52.3885317, Longitude: 4.6388048
Haarlemmermeer: Latitude: 52.3238421, Longitude: 4.715261248876009
Arnhem: Latitude: 51.984257, Longitude: 5.9108573
Zaanstad: Latitude: 52.4689235, Longitude: 4.773783522451712
Amersfoort: Latitude: 52.1562499, Longitude: 5.3896944
Apeldoorn: Latitude: 52.17957375, Longitude: 5.930077460807249
Dordrecht: Latitude: 51.814429, Longitude: 4.6692438
L

###CREATE THE DATAFRAME WITH THE NEW DATA

In [50]:
import datetime as dt
# Drop the last column ('city')
merged_data_Y = merged_data.drop(columns=merged_data.columns[-1])

# Extract the year from the 'date' column and create a new 'Year' column
merged_data_Y['Year'] = merged_data_Y['date'].dt.year

# Drop the original 'date' column
merged_data_Y = merged_data_Y.drop(columns=['date'])

# Group by 'Year' and calculate the mean for all other columns
yearly_averages = merged_data_Y.groupby('Year').mean().reset_index()

# Display the result
print(yearly_averages.head())
# yearly_averages.to_csv('C:/Users/stili/OneDrive/Desktop/yearly_averages_open_meteo.csv', index=False)

   Year  daylight_duration  wind_speed_10m_max
0  1961       44210.539062           21.774876
1  1962       44210.406250           22.598320
2  1963       44210.730469           20.312769
3  1964       44166.367188           20.640671
4  1965       44210.257812           21.647125


###ADD THE NEW DATA TO THE MERGED DATASET AND CREATE THE FINAL (YEARLY WEATHER+SEASON+ YEARLY FROM API)

In [51]:

merged_data_norm = yearly_averages
merged_data_norm=merged_data_norm.drop(columns="daylight_duration")
print(merged_data_norm.to_string())
# merged_data_norm.to_csv('C:/Users/stili/OneDrive/Desktop/yearly_averages_open_meteo_norm.csv', index=False)
final_merged=yearly_data.merge(merged_data_norm, on='Year', how='inner')
print(final_merged.to_string())
final_merged.to_csv('C:/Users/stili/OneDrive/Desktop/Data_sets/Data_Sets/final_yearly_merged_data.csv', index=False)


    Year  wind_speed_10m_max
0   1961           21.774876
1   1962           22.598320
2   1963           20.312769
3   1964           20.640671
4   1965           21.647125
5   1966           21.361628
6   1967           22.624439
7   1968           20.805838
8   1969           20.654068
9   1970           21.892036
10  1971           20.016560
11  1972           20.411646
12  1973           20.879097
13  1974           22.798960
14  1975           21.035250
15  1976           20.967447
16  1977           21.721350
17  1978           21.052631
18  1979           21.601227
19  1980           21.795118
20  1981           21.921700
21  1982           21.121735
22  1983           22.783325
23  1984           21.047876
24  1985           21.302647
25  1986           22.760746
26  1987           20.785089
27  1988           22.759317
28  1989           21.053844
29  1990           22.942909
30  1991           21.112219
31  1992           22.241341
32  1993           22.117634
33  1994      

#PROCESS THE CROP DATASET FROM FAOSTAT

In [54]:
FAOSTAT=pd.read_csv('C:/Users/stili/OneDrive/Desktop/Data_sets/Data_sets/FAOSTAT_data_en_9-29-2024.csv')

#REMOVE ALL THE CROPS THAT HAVE MORE THAN 30% ZERO VALUES
zero_percentage = FAOSTAT.groupby('Item')['Value'].apply(lambda x: (x == 0).sum() / len(x) * 100)
items_to_keep = zero_percentage[zero_percentage <= 30].index

#SAVE THE REDUCED CROP DATASET
FAOSTAT = FAOSTAT[FAOSTAT['Item'].isin(items_to_keep)]

FAOSTAT.to_csv("C:/Users/stili/OneDrive/Desktop/Data_sets/Data_sets/dash/FAOSTAT_nozer.csv", index=False)
FAOSTAT.head()

,Domain,Area,Element,Item,Year,Unit,Value
5,Crops and livestock products,Netherlands (Kingdom of the),Yield,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,14475.0
6,Crops and livestock products,Netherlands (Kingdom of the),Production,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,6000.0
7,Crops and livestock products,Netherlands (Kingdom of the),Yield,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,17735.0
8,Crops and livestock products,Netherlands (Kingdom of the),Production,"Anise, badian, coriander, cumin, caraway, fenn...",1962,t,7000.0
9,Crops and livestock products,Netherlands (Kingdom of the),Yield,"Anise, badian, coriander, cumin, caraway, fenn...",1963,100 g/ha,15711.0
